In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Input Menu

# Trip information
print('----- Seja Bem-vindo ao menú interativo!')
print('Por favor, sempre que possível, escreva em português e inglês.\nObrigado!')
print('Informações da viagem:')
pessoas = int(input('Quantidade de pessoas: '))
diaria = int(input('Quantidade de diárias: '))
UF = str(input('UF de destino: '))
Estado = str(input('Estado de destino: '))
valor = float(input('Gasto planejado: '))
states = [UF, Estado]
print("-----------------------")

# Client requirements

# Facilities
print('Requisições do cliente:')
facilities = []
control = 'Controle'
while (control != ''):
    s = str(input('Facilidade/Instalação exigida: '))
    if len(s) == 0: control = ''
    else: facilities.append(s)

# Environments
locals = []
control = 'Controle'
while (control != ''):
    s = str(input('Lazer desejado (usado para filtragem): '))
    if len(s) == 0: control = ''
    else: locals.append(s)

# Other information
host = str(input('Deseja que o host tenha identificação? (digite: t para sim ou f para não) '))
cama = int(input('Quantidade de camas: '))
quarto = int(input('Quantidade de quartos: '))
banheiro = int(input('Quantidade de banheiros: '))

# Qualities (for recommendation ranking)
control = 'Controle'
qualities = []
while (control != ''):
    s = str(input('Visão geral do bairo (usado para avaliação):  '))
    if len(s) == 0: control = ''
    else: qualities.append(s)
print("-----------------------")

# Acommodation preferences
print('Preferencias da acomodação:')
entry = str(input('Desejar usar pesos predefinidos? (digite: t para sim ou f para não)'))
if entry == "f":
    a = float(input('0 a 10 importancia da avaliação geral: '))
    b = float(input('0 a 10 importancia do número de reviews: '))
    c = float(input('0 a 10 importancia de ser superhost: '))
    d = float(input('0 a 10 importancia da visão geral do bairro: '))
    e = float(input('0 a 10 importancia da data de início do host: '))
    f = float(input('0 a 10 importancia do número de acomodações do host: '))
    g = float(input('0 a 10 importancia de facilidades da casa: '))
    h = float(input('0 a 10 importancia da distância: '))
    i = float(input('0 a 10 importancia do custo total da casa: '))
else:
    # Weights for Perfil 2
    a = 10
    b = 4
    c = 7
    d = 10
    e = 4
    f = 2
    g = 9
    h = 9
    i = 0
w1 = a/(a+b+c+d+e+f+g+h+i)
w2 = b/(a+b+c+d+e+f+g+h+i)
w3 = c/(a+b+c+d+e+f+g+h+i)
w4 = d/(a+b+c+d+e+f+g+h+i)
w5 = e/(a+b+c+d+e+f+g+h+i)
w6 = f/(a+b+c+d+e+f+g+h+i)
w7 = g/(a+b+c+d+e+f+g+h+i)
w8 = h/(a+b+c+d+e+f+g+h+i)
w9 = i/(a+b+c+d+e+f+g+h+i)

----- Seja Bem-vindo ao menú interativo!
Por favor, sempre que possível, escreva em português e inglês.
Obrigado!
Informações da viagem:
Quantidade de pessoas: 2
Quantidade de diárias: 4
UF de destino: RJ
Estado de destino: Rio de Janeiro
Gasto planejado: 4000
-----------------------
Requisições do cliente:
Facilidade/Instalação exigida: piscina
Facilidade/Instalação exigida: pool
Facilidade/Instalação exigida: 
Lazer desejado (usado para filtragem): restaurant
Lazer desejado (usado para filtragem): restaurante
Lazer desejado (usado para filtragem): praia
Lazer desejado (usado para filtragem): beach
Lazer desejado (usado para filtragem): 
Deseja que o host tenha identificação? (digite: t para sim ou f para não) t
Quantidade de camas: 1
Quantidade de quartos: 1
Quantidade de banheiros: 1
Visão geral do bairo (usado para avaliação):  quiet
Visão geral do bairo (usado para avaliação):  calmo
Visão geral do bairo (usado para avaliação):  
-----------------------
Preferencias da acomodação:

In [ ]:
# Load the dataset
path = '/content/drive/MyDrive/base_desafio_MTEch_2021.csv'
df = pd.read_csv(path)

In [ ]:
# Auxiliary function to format prices and taxes
def formatPrices(x):
  return float(x[1:].replace(',', ''))

def formatTaxes(x):
  if x == 0: return 0
  else: float(x[1:].replace(',', ''))

In [ ]:
# First database filter
df.dropna(subset = ['id', 'bairro'], inplace=True)
df['taxa_de_limpeza'] = df['taxa_de_limpeza'].fillna(0)
df['preço'] = df['preço'].apply(formatPrices)
df['taxa_de_limpeza'] = df['taxa_de_limpeza'].apply(formatTaxes)

first_filter = df[(df['noites_mínimas'] <= diaria) & 
              (df['noites_máximas'] >= diaria) & 
              (df['pessoas_inclusas_no_preço'] >= pessoas) &
              (df['host_tem_identidade_verificada'] == 't') &
              (df['camas'] >= cama) &
              (df['banheiros'] >= banheiro) &
              (df['quartos'] >= quarto) &
              (df['estado'].str.contains('|'.join(states), case=False) == True) &
              ((df['facilidades'].str.contains('|'.join(facilities), case=False) == True) | (df['acesso'].str.contains('|'.join(facilities), case=False) == True) ) &
              ((df['visao_geral_bairro'].str.contains('|'.join(locals), case=False) == True) | (df['descrição'].str.contains('|'.join(locals), case=False) == True) ) &
              (df['preço'] * 4  <= valor - 1500 - df['taxa_de_limpeza'])] 
             

In [ ]:
# Distance between points equation
from math import sin, cos, sqrt, atan2, radians 
def distance(p1, p2): 
    R = 6371.0
    lat1 = radians(p1[0]) 
    lon1 = radians(p1[1])
    lat2 = radians(p2[0]) 
    lon2 = radians(p2[1]) 
 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2 
    c = 2 * atan2(sqrt(a), sqrt(1 - a)) 
    distance = R * c 
 
    return distance 

In [ ]:
# OpenCage API to get coordinates by neighborhood (Forward Geocoding)
# Execute request only once !!!!!
import requests
import json

def getDistances(fixed_c, neighborhood_request):
    listOfDistances = []
    API_KEY = 'e06704cf66154accb77df640cd3a06c8'
    url1 = 'https://api.opencagedata.com/geocode/v1/json?q='
    bounds = '&bounds=-44.00000,-24.00000,-42.00000,-22.00000' # Bounded to RJ metropolitan city
    url2 = '&key='

    path = path = url1 + fixed_c + bounds + url2 + API_KEY
    output = requests.get(path).json()
    fixed_lat = output['results'][0]['geometry']['lat']
    fixed_lng = output['results'][0]['geometry']['lng']
    fixed_coord = (fixed_lat, fixed_lng)

    for location in neighborhood_request:
      path = url1 + location + bounds + url2 + API_KEY
      output = requests.get(path).json()
      # Parse output on JSON
      lat = output['results'][0]['geometry']['lat']
      lng = output['results'][0]['geometry']['lng']
      # print(lat, lng)
      listOfDistances.append(distance((lat,lng), fixed_coord))
    return listOfDistances

In [ ]:
# Add the attribute 'Distance' -> distance between Copacabana and each neighbohood
fixed_point = 'Copacabana'
neighborhood = first_filter['bairro'].unique().tolist()
# API Call
listOfDistances = getDistances(fixed_point, neighborhood)

distances = {"bairro" : neighborhood, 
             "Distance": listOfDistances}
dist = pd.DataFrame(distances)
second_filter = first_filter.merge(dist, left_on='bairro', right_on='bairro')

In [ ]:
# Second database filter:
cost_by_km = 3
routes = 2
second_filter = second_filter [(second_filter['preço']* 4 <= 4000 - 1500 - second_filter['Distance'] * cost_by_km * routes
                                                             - second_filter['taxa_de_limpeza'])]
second_filter['preço'] = second_filter['preço'] * 4

In [ ]:
# Auxiliary functions
from datetime import datetime, date
today = date.today()

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days//30)
    
def process(row, lista):
    i = 0
    found = False
    while (i < len(lista) and not found):
      if lista[i] in str(row):
        found = True
      i += 1
    if (found): return 1
    else: return 0
def f(x):    
  return x['preço'] + x['taxa_de_limpeza']

In [ ]:
# Rank algorithm with our final database (9 parameters for rank)

#!pip install scikit-mcda
from scikitmcda.topsis import TOPSIS
from scikitmcda.constants import MAX, MIN, LinearMinMax_, LinearMax_, LinearSum_, Vector_, EnhancedAccuracy_, Logarithmic_ 

criterios = ['avaliação_geral', 'número de reviews', 'superhost', 'visao_geral_bairro', 'inicio_host', 
             'numero_de_acomodacoes_host', 'facilidades', 'Distance', 'preço']
final_db = second_filter.loc[:, criterios]

final_db['superhost'] = final_db['superhost'].apply(lambda x: 1 if x == "t" else 0)
final_db['avaliação_geral'] = final_db['avaliação_geral'].fillna(0)
final_db['visao_geral_bairro'] = final_db['visao_geral_bairro'].apply(lambda row: process(row, qualities))
final_db['inicio_host'] = final_db['inicio_host'].apply(lambda row: days_between(row, str(today)))
final_db['facilidades'] = final_db['facilidades'].apply(lambda row: len(row.split(',')))
final_db['preço'] = second_filter.loc[:,['preço', 'taxa_de_limpeza']].apply(f, axis=1)
second_filter['Current Cost'] = second_filter.loc[:,['preço', 'taxa_de_limpeza']].apply(f, axis=1)

topsis = TOPSIS()
topsis.dataframe(final_db.values, second_filter['id'], final_db.columns)

# Weights
topsis.set_signals([1, 1, 1, 1, 1, 1, 1, -1, -1])
topsis.set_weights_manually([w1, w2, w3, w4, w5, w6, w7, w8, w9])
topsis.decide()

print("TOP 10 ACOMODAÇÕES using TOPSIS")
rank = topsis.df_decision.sort_values(by=['rank'])
rank.head(10)

TOP 10 ACOMODAÇÕES using TOPSIS


,alternatives,performance score,rank
20,8389413,0.724209,1
0,76300,0.714152,2
4,972950,0.646185,3
18,5237812,0.615645,4
21,9857875,0.585485,5
16,3009243,0.553142,6
1,2121969,0.546261,7
7,10810099,0.540983,8
10,14283122,0.531844,9
2,12707963,0.505759,10


In [ ]:
# Once the accomodation is defined, we check if the budget is enough 
best_choice_ID = int(rank.iloc[0]['alternatives'])
best_choice_name = second_filter[second_filter['id'] == best_choice_ID].iloc[0]['nome']
best_choice_cost = second_filter[second_filter['id'] == best_choice_ID].iloc[0]['Current Cost']
best_choice_neighborhood = second_filter[second_filter['id'] == best_choice_ID].iloc[0]['bairro']

# Pontos turísticos: 1) Day Spa at Copacabana Palace(Copacabana, R$1500p/2 pessoas)
                    #2) Praia de Copacabana, 3) Praia de Ipanema, 4) Confeitaria Colombo (Copacabana, R$98 (p/2 pessoas))
tourism_budget = 1500 + 98
tourism_neighborhoods = []
control = 'Controle'
while (control != ''):
    s = str(input('Bairros dos pontos turísticos optativos: '))
    if len(s) == 0: control = ''
    else: tourism_neighborhoods.append(s)

# Here we need to call the API
listDistances = getDistances(best_choice_neighborhood, tourism_neighborhoods)
transport_cost = sumDistances * cost_by_km * routes

print('----- Best choice details -----:')
print(f'Accomodation cost: {best_choice_cost}')
print(f'Transport cost: {transport_cost}')
best_choice_cost += tourism_budget
best_choice_cost += transport_cost

print(f'ID: {best_choice_ID}, Name: {best_choice_name}, Final cost: {best_choice_cost}')

Bairros dos pontos turísticos optativos: Copacabana
Bairros dos pontos turísticos optativos: Ipanema
Bairros dos pontos turísticos optativos: Copacabana
Bairros dos pontos turísticos optativos: Copacabana
Bairros dos pontos turísticos optativos: 
----- Best choice details -----:
Accomodation cost: 756.0
Transport cost: 166.29770695120695
ID: 8389413, Name: Contemporary, front of subway!, Final cost: 2520.297706951207
